<a href="https://colab.research.google.com/github/csbanon/bert-product-rating-predictor/blob/master/Clustering_Preprocessing_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains code to preprocess data for clustering. This updated version includes tagging of negations in the tokens to preserve negative sentiments.

In [ ]:
# Import statements
import numpy as np
import pandas as pd

In the following cell, we access the required data, located in our repository. This url will change depending on the data that will be used for clustering.

We will preprocess reviews with the star ratings in two forms: 
1. as **ground truth**
2. as **predicted by our model**. 


In [ ]:
# Create a dataframe of all reviews (with comments and star ratings only)

# For dataset with rating predictions for all reviews:
# github_url = 'https://raw.githubusercontent.com/csbanon/bert-product-rating-predictor/master/data/predictions_all.csv'

# For dataset with rating predictions only for the test dataset
# github_url = 'https://raw.githubusercontent.com/csbanon/bert-product-rating-predictor/master/data/predictions_test.csv'

# For dataset with ground truth star ratings and reviews:
github_url = 'https://raw.githubusercontent.com/csbanon/bert-product-rating-predictor/master/data/reviews_comments_stars.csv'

df = pd.read_csv(github_url)

To preprocess reviews from the dataset containing the ground truth star ratings, minor modifications are made, as described in the relevant comments. 
Please note: These modifications are only made for the purpose of code reusability. They do not modify the nature of the data itself.

In [ ]:
# Execute this cell if preprocessing data from the predicted datasets
# df = df[['review', 'prediction']] # column names corresponding to predicted data csv files
# df.head()

In [ ]:
# Execute this cell only if preprocessing ground truth dataset
df = df[['comment','stars']] # column names corresponding to ground truth csv file

# Renaming columns to match column names of predicted datasets for code reusability
df.rename(columns={"comment": "review", "stars": "prediction"}, inplace=True)

df.head()

,review,prediction
0,I could sit here and write all about the specs...,5
1,A very reasonably priced laptop for basic comp...,4
2,"This is the best laptop deal you can get, full...",5
3,A few months after the purchase....It is still...,5
4,BUYER BE AWARE: This computer has Microsoft 10...,1


In [ ]:
# Create review dataframes for each category of star rating
df1 = df[df['prediction']==1]
df2 = df[df['prediction']==2]
df3 = df[df['prediction']==3]
df4 = df[df['prediction']==4]
df5 = df[df['prediction']==5]

df1.head()

,review,prediction
4,BUYER BE AWARE: This computer has Microsoft 10...,1
14,Why Acer install this version of Windows 10 wi...,1
15,As I write this I am on the phone with Microso...,1
23,"Just abominable, at the absolute worst time in...",1
26,Beware of Windows 10s. I’ve been on live chat ...,1


In [ ]:
# Required modules
!pip install Unidecode

# Following modules are required if expanding contractions using pycontractions.
# !sudo apt install openjdk-8-jdk
# !sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
# !pip install language-check
# !pip install pycontractions
# !pip install gensim

In [ ]:
# More imports
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer # May import WordPunctTokenizer & word_tokenize for other approaches
# from nltk.sentiment.util import mark_negation
# from pycontractions import Contractions

import string
import unidecode
import re
import unidecode

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Convert accented characters to ascii
def remove_accents(text):
  text = unidecode.unidecode(text)
  return text

In [ ]:
# Remove numeric characters
def remove_nums(text):
  text = re.sub("\d+", " ", text)
  return text

In [ ]:
# Contractions initialization
# cont = Contractions(api_key="glove-twitter-100")
# cont.load_models()

In [ ]:
# Expand any contractions
# def expand_contractions(text):
#   text = list(cont.expand_texts([text], precise=False))
#   textToStr = ' '.join([str(elem) for elem in text])
#   return textToStr

In [ ]:
# Tokenizer instantiation: only consider alphabetic characters for tokenization
tokenizer = RegexpTokenizer(r"\w+[a-zA-Z]")
def get_tokens(text):
  # tokens = word_tokenize(text.lower())
  #  split into words by white space
  words = text.replace("'","")
  tokens = tokenizer.tokenize(words.lower())
  return tokens

In [ ]:
# Mark negations: if a word is preceeded by a form of "not" or "no", it is flagged as a negated word.
negations_list = ["no", "not", "n't",
"shouldn't", "shouldnt",
"couldn't", "couldnt",
"wouldn't", "wouldnt",
"hadn't", "hadnt",
"didn't", "didnt",
"isn't", "isnt",
"won't", "wont",
"can't", "cant",
"doesn't", "doesnt",
"aren't", "arent",
"wasn't", "wasnt",
"haven't", "havent",
"hasn't", "hasnt",
"don't", "dont",
"weren't", "werent",
"mightn't", "mightnt",
"needn't", "neednt"]

def mark_neg(text):
  for i in range(0,len(text)):
    if i == len(text) - 1:
      break
    else:
      if text[i] in negations_list:
        text[i+1] = 'NOT_'+text[i+1]
  return text

In [ ]:
# Remove stop words
def remove_stopwords(text):
  stoplist = stopwords.words('english')
  stoplist = set([word for word in stoplist])
  words = [w for w in text if (w not in stoplist and w not in negations_list)]
  return words

In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer() 
def lemmatize(text):
  lem_text = [lemmatizer.lemmatize(i) for i in text]
  return lem_text

In [ ]:
pd.options.mode.chained_assignment = None  # turns off Pandas warning when processing all dataframes

# Perform preprocessing for each category of star rating
df_list = [df1, df2, df3, df4, df5]

for frame in df_list:
  frame['review'] = frame['review'].apply(lambda x: remove_accents(x))
  frame['review'] = frame['review'].apply(lambda x: remove_nums(x))
  # frame['review'] = frame['review'].apply(lambda x: expand_contractions(x)) # runtime too long for this step 
  frame['review'] = frame['review'].apply(lambda x: get_tokens(x)) # tokenizes and convert to lowercase
  frame['review'] = frame['review'].apply(lambda x: mark_neg(x))
  frame['review'] = frame['review'].apply(lambda x: remove_stopwords(x))
  frame['review'] = frame['review'].apply(lambda x: lemmatize(x))

In [ ]:
# Test
df1.head()

,review,prediction
4,"[buyer, aware, computer, microsoft, scam, soft...",1
14,"[acer, install, version, window, forced, mode,...",1
15,"[write, phone, microsoft, attempting, get, mul...",1
23,"[abominable, absolute, worst, time, life, poss...",1
26,"[beware, window, ive, live, chat, microsoft, h...",1


In [ ]:
# Test: Inspect one set of tokens for one review comment
# stringg = df2.iloc[0:1,0:1]
# stringg.to_csv('sample.csv')

In [ ]:
# Test
df5.head()

,review,prediction
0,"[could, sit, write, spec, computer, already, d...",5
2,"[best, laptop, deal, get, full, stop, touchesc...",5
3,"[month, purchase, still, running, good, bought...",5
5,"[update, labtop, completely, died, near, end, ...",5
7,"[run, amazing]",5


In [ ]:
# Execute this cell only if preprocessing ground truth dataset.
# Renaming "prediction" column to "stars" because these are ground truth ratings, not predicted ratings.
for frame in df_list:
  frame.rename(columns={"prediction":"stars"}, inplace=True)

In [ ]:
# Create composite dataframe of tokens for all reviews, for all star ratings
mega_df = pd.concat(objs=[df1,df2,df3,df4,df5])
mega_df.head()

,review,stars
4,"[buyer, aware, computer, microsoft, scam, soft...",1
14,"[acer, install, version, window, forced, mode,...",1
15,"[write, phone, microsoft, attempting, get, mul...",1
23,"[abominable, absolute, worst, time, life, poss...",1
26,"[beware, window, ive, live, chat, microsoft, h...",1


The following code is used to save tokens as .csv files. The naming convention changes depending on the data that has been preprocessed.

In [ ]:
# For clustering using predictions made on the entire dataset:

# mega_df.to_csv('tokens_all_predictions.csv') # Save tokens from all reviews, for all predicted ratings
# for idx, frame in enumerate(df_list): # Save tokens for each category of predicted star rating [1-5]
#   filename = 'tokens_all_predictions_'+str(idx+1)+'.csv'
#   frame.to_csv(filename)

In [ ]:
# For clustering using predictions made only on test dataset:

# mega_df.to_csv('tokens_test_predictions.csv') # Save tokens from all reviews, for all predicted ratings
# for idx, frame in enumerate(df_list): # Save tokens for each category of predicted star rating [1-5] 
#   filename = 'tokens_test_predictions_'+str(idx+1)+'.csv'
#   frame.to_csv(filename)

In [ ]:
# For clustering using ground truth dataset:

mega_df.to_csv('tokens_true_ratings_all.csv') # Save tokens from all reviews, for all true ratings
for idx, frame in enumerate(df_list): # Save tokens for each category of star rating [1-5] 
  filename = 'tokens_true_ratings_'+str(idx+1)+'.csv'
  frame.to_csv(filename)

#References:
1. https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
2. https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f
3. https://pypi.org/project/pycontractions/
4. https://machinelearningmastery.com/clean-text-machine-learning-python/
5. https://web.stanford.edu/~jurafsky/slp3/4.pdf (pg 8)